# Mapping real data to Wofost input/output 

In [156]:
import sys, os.path
import yaml
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from IPython.display import display
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 250)

import pcse
from pcse.models import Wofost71_PP
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from pcse.fileinput import YAMLCropDataProvider
# from pcse.util import WOFOST71SiteDataProvider, DummySoilDataProvider
from progressbar import printProgressBar
from pcse.fileinput import CABOFileReader
from pcse.engine import Engine

import utils
from ast import literal_eval

### Get default data

In [2]:
## Retrieve data from default param files
data_dir = os.path.join(os.getcwd(), 'default_data') # Rogerio's data
crop_file_name = "crop.cab"
soil_file_name = "soil.cab" # Must be a CABO file
site_file_name = "site.cab" # Must be a CABO file
agro_file_name = "agro.yaml"# Must be a YAML file
config_file_name = "WLP_NPK.conf" # Water-limited and nutrient-limited production simulation

soild = CABOFileReader(os.path.join(data_dir, soil_file_name))
sited = CABOFileReader(os.path.join(data_dir, site_file_name))
cropd = CABOFileReader(os.path.join(data_dir, crop_file_name))
config = os.path.join(data_dir, config_file_name)

### Override some of the default data


In [151]:
# Define crop and crop variety
crop_name = 'wheat'
variety_name = 'Winter_wheat_101'
## Override crop data
## TODO:  get missing 'CO2' variable when using this, created the issue on the below repo.
# cropd = YAMLCropDataProvider() # pulls from https://github.com/ajwdewit/WOFOST_crop_parameters
# cropd.set_active_crop(crop_name, variety_name)
# cropd.keys()

**Override agromanagement data**

In [153]:
# Overriding agromanagement
campaign_start_date = '2017-01-01'
emergence_date = "2017-03-31"
harvest_date = "2017-08-11"
max_duration = 200

agro_variety_name = 'winter-wheat'
agro_yaml = """
- {start}:
    CropCalendar:
        crop_name: {cname}
        variety_name: {vname}
        crop_start_date: {startdate}
        crop_start_type: emergence
        crop_end_date: {enddate}
        crop_end_type: harvest
        max_duration: {maxdur}
    TimedEvents: null
    StateEvents: null
""".format(cname=crop_name, vname=variety_name, 
           start=campaign_start_date, startdate=emergence_date, 
           enddate=harvest_date, maxdur=max_duration)
agromanagement = yaml.safe_load(agro_yaml)
print(agro_yaml)


- 2017-01-01:
    CropCalendar:
        crop_name: wheat
        variety_name: Winter_wheat_101
        crop_start_date: 2017-03-31
        crop_start_type: emergence
        crop_end_date: 2017-08-11
        crop_end_type: harvest
        max_duration: 200
    TimedEvents: null
    StateEvents: null



**Override soil data**

Real soil data available only for the below variables, so only override those:

- SMW :  soil moisture content at wilting point [cm3/cm3]
- SMFCF :  soil moisture content at field capacity [cm3/cm3]
- K0 : hydraulic conductivity of saturated soil [cm day-1]
- SOPE  : maximum percolation rate root zone[cm day-1]
- KSUB : maximum percolation rate subsoil [cm day-1]

In [157]:
soil_data_path = 'actual_data/soil/soils_locations.csv' #soils_1stDraft.csv'
soil_cols = ['SMW', 'SMFCF', 'K0', 'SOPE', 'KSUB', 'center'] # center: [longitude, latitude]

In [160]:
def get_soil_data(soil_data_path, soil_cols):
    df_soil = pd.read_csv(soil_data_path, usecols=soil_cols)
    df_soil['center'] = df_soil['center'].apply(literal_eval)
    df_soil['longitude'] = df_soil['center'].apply(lambda x: x[0])
    df_soil['latitude'] =  df_soil['center'].apply(lambda x: x[1])
    return df_soil.drop(columns=['center'])

In [161]:
df_soil = get_soil_data(soil_data_path, soil_cols)
df_soil.head()

,SMW,SMFCF,K0,SOPE,KSUB,longitude,latitude
0,0.089795,0.189621,1.357097,148.254720,99.734993,-94.012500,36.737500
1,0.156155,0.264972,0.426985,130.503168,87.793040,-94.012500,36.737500
2,0.114223,0.219987,0.886029,138.696192,93.304711,-86.904167,32.829166
3,0.156155,0.264972,0.426985,130.503168,87.793040,-86.904167,32.829166
4,0.267157,0.399190,0.060149,74.432160,50.072544,-86.904167,32.829166


## Test WOFOST on one data point

In [162]:
# Override soild
soil_row = df_soil.loc[0]
for col in soil_cols[:-1]:
    soild[col] = soil_row[col]
# Run WOFOST
latitude, longitude = soil_row['latitude'], soil_row['longitude']
wdp = NASAPowerWeatherDataProvider(latitude=latitude, longitude=longitude)
params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)
wofost = Engine(params, wdp, agromanagement, config) #WLP_NPK

In [163]:
wofost.run_till_terminate()
r = wofost.get_summary_output()
r

[{'DVS': 2.0,
  'LAIMAX': 0.43441937790317015,
  'TAGP': 2059.221540800439,
  'TWSO': 472.4450240397172,
  'TWLV': 540.1620697272227,
  'TWST': 1046.614447033499,
  'TWRT': 534.0624012598115,
  'CTRAT': 4.128108178866169,
  'RD': 97.60000000000014,
  'DOS': None,
  'DOE': datetime.date(2017, 3, 31),
  'DOA': datetime.date(2017, 6, 2),
  'DOM': datetime.date(2017, 7, 14),
  'DOH': datetime.date(2017, 8, 11),
  'DOV': None}]

In [174]:
r[0]

{'DVS': 2.0,
 'LAIMAX': 0.43441937790317015,
 'TAGP': 2059.221540800439,
 'TWSO': 472.4450240397172,
 'TWLV': 540.1620697272227,
 'TWST': 1046.614447033499,
 'TWRT': 534.0624012598115,
 'CTRAT': 4.128108178866169,
 'RD': 97.60000000000014,
 'DOS': None,
 'DOE': datetime.date(2017, 3, 31),
 'DOA': datetime.date(2017, 6, 2),
 'DOM': datetime.date(2017, 7, 14),
 'DOH': datetime.date(2017, 8, 11),
 'DOV': None}

## Map Wofost output to actual yield_data

In [166]:
def process_yield_data(yield_data_path):
    cols = ['County', 'Value', 'Year', 'State']
    yield_data = pd.read_csv(yield_data_path, usecols=cols)
    yield_data['State'] = utils.us_state_abbrev(yield_data['State'])
    
    # Convert Actual Yield Data in bushels/acre to kg/ha
    conversion_rate = 67.2511 # 1 bu/acre to kg/ha # http://www.kylesconverter.com/area-density/bushels-per-acre-to-kilograms-per-hectare
    yield_data['Value'] = yield_data['Value'] * conversion_rate
    return yield_data

yield_data_path = 'actual_data/yield_usda/wheat_irrigated_country_annual.csv'
yield_data = process_yield_data(yield_data_path)

In [167]:
def process_county_data(county_coords_path):
    cols = ['county', 'latitude', 'longitude', 'state']
    county_coords = pd.read_csv(county_coords_path, usecols=cols)
    county_coords['county'] = county_coords['county'].str.upper()
    county_coords = county_coords.dropna()
    county_coords = county_coords.drop_duplicates()
    return county_coords

county_coords_path = 'actual_data/others/Geocodes_USA_with_Counties.csv'
county_coords = process_county_data(county_coords_path )
county_coords.head()

,state,latitude,longitude,county
0,NY,40.81,-73.04,SUFFOLK
2,PR,18.16,-66.72,ADJUNTAS
4,PR,18.43,-67.15,AGUADILLA
7,PR,18.18,-66.98,MARICAO
10,PR,18.45,-66.73,ARECIBO


In [168]:
clean_yield_data = pd.merge(yield_data, county_coords,  how='left', left_on=['County','State'], right_on = ['county','state'])
clean_yield_data = clean_yield_data.drop(columns=['county', 'state'])
clean_yield_data = clean_yield_data.dropna()
clean_yield_data = clean_yield_data.drop_duplicates()
clean_yield_data.describe()

,Year,Value,latitude,longitude
count,258030.000000,258030.000000,258030.000000,258030.000000
mean,1980.708708,3643.382307,40.096418,-110.791067
std,14.962539,1555.262989,4.725380,8.211198
min,1929.000000,0.000000,25.920000,-124.140000
25%,1974.000000,2421.039600,36.580000,-118.270000
50%,1982.000000,3362.555000,40.050000,-111.670000
75%,1990.000000,4788.278320,43.880000,-103.820000
max,2007.000000,9146.149600,48.990000,-94.070000


In [169]:
lon_eps = 1 # 1 degree --> 87.87018 km
lat_eps = 0.5 # 1 degree --> 111.045 km
lat_data = clean_yield_data[abs(clean_yield_data['latitude']-latitude) <= lat_eps]
lat_data.head()
lon_data = lat_data[abs(lat_data['longitude']-longitude) <= lon_eps]
len(set(lon_data['County']))

1

In [170]:
coords = clean_yield_data[['latitude', 'longitude']]
coords.head()

,latitude,longitude
0,37.86,-121.64
1,37.78,-121.88
2,37.84,-121.97
3,37.99,-121.81
4,38.07,-121.62


In [171]:
soil_subset = pd.merge(df_soil, coords, on=['latitude', 'longitude'], how='left')
soil_subset.describe()

,SMW,SMFCF,K0,SOPE,KSUB,longitude,latitude
count,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,0.129397,0.257961,0.887032,110.882420,74.593628,-96.088458,40.289006
std,0.046879,0.057563,0.745602,26.439519,17.786586,14.970418,5.928688
min,0.022273,0.095743,0.009053,46.780704,31.470655,-123.787500,24.595833
25%,0.096445,0.219987,0.410936,91.756992,61.727431,-109.704167,35.820833
50%,0.127934,0.264972,0.679608,109.081824,73.382318,-94.854167,40.937500
75%,0.156345,0.298316,1.158441,130.503168,87.793040,-82.820834,44.970833
max,0.318037,0.445473,4.405855,189.219840,127.293347,-62.062500,49.120833


# WOFOST Input/Output Matrix using real data
Input:
- soil_subset: soil data that has the same coordinates as the yield_data
- weather: default data provider using latitude, longitude from soil_subset
- cropd: constant
- sited: constant
- agro_yaml: constant (careful with dates, some dates don't have weather data!)

Output:
- clean_yield_data: output yield in kg/ha

In [173]:
## Retrieve data from default param files
data_dir = os.path.join(os.getcwd(), 'default_data') # Rogerio's data
crop_file_name = "crop.cab"
soil_file_name = "soil.cab" # Must be a CABO file
site_file_name = "site.cab" # Must be a CABO file
agro_file_name = "agro.yaml"# Must be a YAML file
config_file_name = "WLP_NPK.conf" # Water-limited and nutrient-limited production simulation

soild = CABOFileReader(os.path.join(data_dir, soil_file_name))
sited = CABOFileReader(os.path.join(data_dir, site_file_name))
cropd = CABOFileReader(os.path.join(data_dir, crop_file_name))
config = os.path.join(data_dir, config_file_name)
# Define crop and crop variety
crop_name = 'wheat'
variety_name = 'Winter_wheat_101'
## Override crop data
## TODO:  get missing 'CO2' variable when using this, created the issue on the below repo.
# cropd = YAMLCropDataProvider() # pulls from https://github.com/ajwdewit/WOFOST_crop_parameters
# cropd.set_active_crop(crop_name, variety_name)
# cropd.keys()
# Overriding agromanagement
campaign_start_date = '2017-01-01'
emergence_date = "2017-03-31"
harvest_date = "2017-08-11"
max_duration = 200

agro_variety_name = 'winter-wheat'
agro_yaml = """
- {start}:
    CropCalendar:
        crop_name: {cname}
        variety_name: {vname}
        crop_start_date: {startdate}
        crop_start_type: emergence
        crop_end_date: {enddate}
        crop_end_type: harvest
        max_duration: {maxdur}
    TimedEvents: null
    StateEvents: null
""".format(cname=crop_name, vname=variety_name, 
           start=campaign_start_date, startdate=emergence_date, 
           enddate=harvest_date, maxdur=max_duration)
agromanagement = yaml.safe_load(agro_yaml)
print(agro_yaml)


- 2017-01-01:
    CropCalendar:
        crop_name: wheat
        variety_name: Winter_wheat_101
        crop_start_date: 2017-03-31
        crop_start_type: emergence
        crop_end_date: 2017-08-11
        crop_end_type: harvest
        max_duration: 200
    TimedEvents: null
    StateEvents: null



In [ ]:
df_results = pd.DataFrame()
nEpochs = len(soil_subset)
for i in range(len(soil_subset)):
    # New location: new soil, new coordinates
    soil_row = df_soil.loc[i]
    for col in soil_cols[:-1]:
        soild[col] = soil_row[col]
    # Get 
    latitude, longitude = soil_row['latitude'], soil_row['longitude']
    try:
        wdp = NASAPowerWeatherDataProvider(latitude=latitude, longitude=longitude)
        params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)
        wofost = Engine(params, wdp, agromanagement, config) #WLP_NPK
        wofost.run_till_terminate()
        r = wofost.get_summary_output()
        df_results = df_results.append(r[0], ignore_index=True)
    except Exception as e:
        print('Exception at ({}, {}):'.format(latitude, longitude), e)
        continue
   

Exception at (43.82916626740311, -123.37083355985): No weather data for 2017-07-18.
Exception at (43.82916626740311, -123.37083355985): No weather data for 2017-07-18.
Exception at (43.82916626740311, -123.37083355985): No weather data for 2017-07-18.
Exception at (34.24583297240311, -85.72916704375): No weather data for 2017-03-24.
Exception at (35.79583296620311, -80.87083372985): No weather data for 2017-05-25.
Exception at (35.79583296620311, -80.87083372985): No weather data for 2017-05-25.
Exception at (35.79583296620311, -80.87083372985): No weather data for 2017-05-25.
Exception at (35.67083296670311, -83.75416705165): No weather data for 2017-08-06.
Exception at (35.67083296670311, -83.75416705165): No weather data for 2017-08-06.
Exception at (35.67083296670311, -83.75416705165): No weather data for 2017-08-06.


In [ ]:
df_results